In [1]:
from tkinter import *
from tkinter import messagebox
from tkinter import font
from tkinter import filedialog
from tkinter import ttk
from functools import partial
import os
import pandas as pd
import numpy as np
import time
import os.path as path
import pickle
import Funciones as fn
import NN as rn
import RF as rf

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Metodos Para Guardar y Cargar arhivos

## guardar Log_Historico_Cargados

In [3]:
#Método que sirve para escribir en el LOG_HISTORICO_CARGADOS.TXT

def guardarLogCargados(listaFeaturesCargada, modeloCargadoRN, modeloCargadoRF):
    
    fecha = (time.strftime("%Y_%m_%d")) #Fecha en el formato YYYY_MM_DD
    hora = time.strftime("%H:%M:%S") #Hora en el formato H:M:S
    
    #Ruta del .txt
    rutaFichero= os.getcwd() #Sacamos la ruta donde estamos ejecutando el script
    fileName="LOG_HISTORICO_CARGADOS.txt"
    completeName=rutaFichero + "/" + fileName #Ruta y nombre del archivo donde vamos a escribir
    
    #Si el fichero no existe, le crea.
    if (path.exists(completeName) == False):
        archivo = open(completeName, 'w')
    else:
        archivo = open(completeName, 'a')
    
    #Escribimos en el archivo
    archivo.write("Fecha: " + fecha + " a las " + hora)
    archivo.write("\n")
    archivo.write("Ruta Modelo (RN) Cargado: " + modeloCargadoRN)
    archivo.write("\n") 
    archivo.write("Ruta Modelo (RF) Cargado: " + modeloCargadoRF)
    archivo.write("\n") 
    archivo.write("Ruta Lista características Cargada: " + listaFeaturesCargada)
    archivo.write("\n") 
    archivo.write("\n") 
    
    archivo.close()

## Cargar log_historico_cargados

In [4]:
#Método para cargar el archivo LOG_HISTORICO_CARGADOS.TXT

def cargarLogCargados(ruta_fichero):
    listaUltimasLineas = []
    
    #Abrimos el archivo para escribir en él
    archivo = open(ruta_fichero, 'r')

    #Tratamiento para coger las ultimas lineas
    lineas = archivo.readlines()
    lineaCaracteristicas = lineas[len(lineas)-2] 
    lineaRF = lineas[len(lineas)-3] 
    lineaRN = lineas[len(lineas)-4] 

    #Tratamiento de las lineas ledias
    lineaCaracteristicas=lineaCaracteristicas[-49:]
    lineaCaracteristicas=lineaCaracteristicas[:-1]
    
    lineaRF=lineaRF[-32:]
    lineaRF=lineaRF[:-1]
    
    lineaRN=lineaRN[-32:]
    lineaRN=lineaRN[:-1]

    listaUltimasLineas.append(lineaCaracteristicas)
    listaUltimasLineas.append(lineaRF)
    listaUltimasLineas.append(lineaRN)
    archivo.close()
    
    return listaUltimasLineas

## cargamos los modelos 

In [5]:
def cargarModelo(ruta_Fichero):

    loaded_model = pickle.load(open(ruta_Fichero, 'rb'))

    return loaded_model

## guardamos los modelos creados

In [6]:
def guardarModelo(modelo, nombreModelo):

    #Sacamos El dia y la hora actual para tener un control
    dataFile = (time.strftime("%Y-%m-%d___%H-%M-%S"))
    fileName="Modelo_%s_" % dataFile + nombreModelo 
    pickle.dump(modelo, open(fileName, 'wb'))

## guardar un archivo con la lista de parametros elegidos en el modelo creado

In [7]:
#Método para escribir en un archivo la listaParametrosElegidos

def guardarArchivoListaParametrosElegidos(listaParametrosElegidos, resultadosRedNeuronal, resultadosRandomForest):
    
    #Sacamos El dia y la hora actual para tener un control
    dataFile = (time.strftime("%Y-%m-%d___%H-%M-%S"))
    
    rutaFichero= os.getcwd() #Sacamos la ruta donde estmos ejecutando el script
    fileName="Modelo_%s_caracteristicas.txt" % dataFile
    completeName=rutaFichero + "/" + fileName #Ruta y nombre del archivo donde vamos a escribir

    #Abrimos el archivo para escribir en él
    archivo = open(completeName, 'w')
   
    #print(listaParametrosElegidos[len(listaParametrosElegidos)-1].get())
    #print(listaParametrosElegidos[len(listaParametrosElegidos)-2].get())

    #Escribimos en el archivo la listaParametrosElegidos
    for i in range(len(listaParametrosElegidos)-2):
        archivo.write(listaParametrosElegidos[i])
        archivo.write("\n") 
    
    #Numero de capas
    archivo.write(listaParametrosElegidos[i+1])
    archivo.write("\n")
    
    #Numero de nodos por capa
    archivo.write(listaParametrosElegidos[i+2])
    archivo.write("\n")
        
    archivo.write("resultado_RN: " +resultadosRedNeuronal + "%")
    archivo.write("\n") 
    archivo.write("resultado_RF: " +resultadosRandomForest + "%")
    
    archivo.close()

    return fileName

## Cargar un archivo con la lista de parametros elegidos en el modelo creado

In [8]:
#Método para leer de un archivo la listaParametrosElegidos

def cargarArchivoListaParametrosElegidos(ruta_fichero):
    listaParametrosElegidos = []
    
    #Abrimos el archivo para leer de él
    archivo = open(ruta_fichero, 'r')
    lineas=archivo.readlines()
    contador = 0

    #Leemos y guardamos en la listaParametrosElegidos los features
    for linea in lineas:
        if(contador != len(lineas)-1):
            listaParametrosElegidos.append(linea[:-1]) #Evitamos que guarde el salto de linea
            contador+=1
        else:
            listaParametrosElegidos.append(linea)
        
    archivo.close()

    return listaParametrosElegidos

# Creacion Ventanas Emergentes

## Ventana Emergente Cargar Modelo

In [9]:
#Ventana Emergente para cuando pulsamos Cargar Modelo

def cargarModeloEmerg(ventanaRaiz):
    
    ruta_fichero_RN = cargaModeloRN() #Obtenemos el modelo de red neuronal
    ruta_fichero_RF = cargaModeloRF() #Obtenemos el modelo de random forest
    
    ruta_fichero_Features = ruta_fichero_RN[:-2] #Le Quitamos el _RN
    ruta_fichero_Features = ruta_fichero_Features + "caracteristicas.txt" #Obtenemos la lista de features

    
    listaParametrosElegidos=cargarArchivoListaParametrosElegidos(ruta_fichero_Features) #Cargará la listaParametrosElegidos del .txt
        
    mlp = cargarModelo(ruta_fichero_RN) #Obtenemos el modelo de la red neuronal
    randomForest = cargarModelo(ruta_fichero_RF) #Obtenemos el modelo del Random Forest
    
    valor=messagebox.askokcancel("Cargar Modelo", "Su modelo se ha cargado correctamente. ¿Desea ver los resultados?")
    
    #Escribimos en el LOG_HISTORICO_CARGADOS que hemos cargado un modelo
    guardarLogCargados(ruta_fichero_Features,ruta_fichero_RN,ruta_fichero_RF)
    
    #Si pulsamos aceptar en la ventana emergente nos enseñara el resultado con dicho modelo
    if valor==True: 
        destruirVent(ventanaRaiz)
        ventanaResultadosModeloCargar(ventanaPostParametros, listaParametrosElegidos)
    else:
        ventanaInicio(ventanaRaiz)

## Ventana emergente guardar Modelo

In [10]:
#Ventana Emergente para cuando hemos clickado en guardar modelo

def guardarModeloEmerg(listaParametrosElegidos, mlp, randomForest, resultadosRedNeuronal, resultadosRandomForest):
    guardarArchivoListaParametrosElegidos(listaParametrosElegidos, resultadosRedNeuronal, resultadosRandomForest)
    guardarModelo(mlp, str("RN"))
    guardarModelo(randomForest, str("RF"))
    
    messagebox.showinfo("Guardar Modelo", "Se ha guardado correctamente")

## Ventana Emergente Acerca de

In [11]:
#Ventana Emergente en el apartado "Acerca de" del menú

def acercaDeEmerg():
    messagebox.showinfo("Acerca de", "Proyecto realizado por Dámaso Sánchez y Eduardo Rodriguez")

## Ventana Emergente Ayuda

In [12]:
#Ventana Emergente en el apartado "Ayuda" del menú

def ayudaEmerg():
    messagebox.showinfo("Ayuda", "Memoria adjunta donde se explica el funcionamiento del software.")

## Ventana Emergente Salir

In [13]:
#Ventana Emergente en el apartado "Salir" del menú

def salirEmerg(ventana):
    valor=messagebox.askokcancel("Salir", "¿Deseas Salir de la aplicación?")
        
    #Si hemos clickado en aceptar saldremos de la aplicación
    if valor==True:
        destruirVent(ventana)

## Creación del menu de opciones.

In [14]:
#Método donde creamos el menú en la ventana que corresponda

def barraMenu(ventana):
    barraMenu=Menu(ventana)
    ventana.config(menu=barraMenu)
    
    barraMenu.add_cascade(label="Salir", command=lambda:salirEmerg(ventana))
    barraMenu.add_cascade(label="Ayuda", command=ayudaEmerg)
    barraMenu.add_cascade(label="Acerca de", command=acercaDeEmerg)
        

## Ocultar una ventana (Sin destruirla)

In [15]:
#Método para ocultar una ventana

def ocultarVent(ventana):
    ventana.withdraw()

## Ventana Principal del software

In [16]:
#Ventana donde mostramos las 3 caracteristicas principales del software:

def ventanaInicial(raiz):
    ocultarVent(raiz)
    
    #Creamos la nueva ventana con sus caracteristicas correspondientes
    ventanaRaiz=Toplevel(raiz) #Nueva ventana
    ventanaRaiz.iconbitmap("imagen.ico") #Ponemos una imagen a nuestro gusto para la ventana
    ventanaRaiz.resizable(False,False) #Hacemos que la ventana no pueda variar su tamaño.
    ventanaRaiz.geometry("650x500") #Damos 650 de ancho y 500 de alto
    barraMenu(ventanaRaiz)
    

    #Creamos el encabezado de la ventana 
    tituloRaiz=Label(ventanaRaiz,text="Predictor de riesgos", font=("arial bold", 24), anchor=CENTER,pady=29)

    #Cuadramos el titulo
    tituloRaiz.pack()

    #Creamos tres frames donde estarán los botones principales y los configuramos
    frameA = Frame(ventanaRaiz) 
    frameB = Frame(ventanaRaiz)
    frameC = Frame(ventanaRaiz)

    frameA.config(width = "182", height = "127") #Damos tamaño al frame y por tanto a la raiz
    frameB.config(width = "182", height = "127") #Damos tamaño al frame y por tanto a la raiz
    frameC.config(width = "182", height = "127") #Damos tamaño al frame y por tanto a la raiz

    #Modificamos los marcos de los frames
    frameA.config(bd=10)
    frameA.config(relief="groove")

    frameB.config(bd=10)
    frameB.config(relief="groove")

    frameC.config(bd=10)
    frameC.config(relief="groove")

    #Colocamos los Frames
    frameA.pack(fill = "x")
    frameB.pack(fill = "x")
    frameC.pack(fill = "x")

    #Cambiamos el tipo del cursor cuando estemos en los frames
    frameA.config(cursor="hand2")
    frameB.config(cursor="hand2")
    frameC.config(cursor="hand2")
    
    #COMIENZA LA FUNCIONALIDAD DE LOS BOTONES
    
    
    #Lo usamos para saber cuando "solicitar riesgo" debe estar activo
    
    rutaFichero= os.getcwd() #Sacamos la ruta donde estamos ejecutando el script
    fileName="LOG_HISTORICO_CARGADOS.txt"
    completeName=rutaFichero + "/" + fileName #Ruta y nombre del archivo

    #Widget para el boton de Cargar modelo
    buttonCargar = Button(frameA, text = "Creación del modelo", bg = "green",fg = "white",font = arial, command=lambda:botonCreacion(ventanaRaiz))
    buttonCargar.config(width = 35, height = 1)
    buttonCargar.place(relx=0.5, rely=0.5, anchor=CENTER)

    #Widget para el boton de Creacion de modelo
    buttonCreate = Button(frameB, text = "Carga del modelo", bg = "green",fg = "white",font = arial, command=lambda:cargarModeloEmerg(ventanaRaiz))
    buttonCreate.config(width = 35, height = 1)
    buttonCreate.place(relx=0.5, rely=0.5, anchor=CENTER)

    #Widget para el boton de Solicitar riesgo
    #Si no hay un archivo historico significa que no has cargado ningun modelo y por tanto no puedes solicitar riesgo
    if (path.exists(completeName) == False): 
        buttonRiesgo = Button(frameC, text = "Solicitar riesgo", bg = "black",font = arial,command=lambda:ventPreFormulario(ventanaRaiz))
        buttonRiesgo.config(width = 35, height = 1, state=DISABLED)
        buttonRiesgo.place(relx=0.5, rely=0.5, anchor=CENTER)
    else:
        buttonRiesgo = Button(frameC, text = "Solicitar riesgo", bg = "green",fg = "white",font = arial,command=lambda:ventPreFormulario(ventanaRaiz))
        buttonRiesgo.config(width = 35, height = 1, state=NORMAL)
        buttonRiesgo.place(relx=0.5, rely=0.5, anchor=CENTER)


# Raiz y Frames iniciales

In [17]:
#Creamos la raiz principal del software y la ocultamos para que se esté ejecutando en segundo plano

raiz = Tk()
ocultarVent(raiz)
raiz.title("Predictor de Riesgos") #Titulo de la ventana raiz

#Creamos un nuevo tipo de letra que será el que usemos en todo el software
arial = font.Font(family="arial bold", size=16, weight = font.BOLD)

#Mostramos la ventana Princiapal que es el punto de origen de la aplicación.
ventanaInicial(raiz)

## Mostrar una ventana

In [18]:
#Metodo para mostrar una ventana

def mostrarVent(ventana):
    ventana.deiconify()

## Destruir una ventana

In [ ]:
#Metodo para destruir una ventana

def destruirVent(ventana):
    ventana.destroy()

# Widgets (Ventanas y botones) y Funciones Auxiliares

## Ventana de creación de un modelo al pulsar el botón "Creacion modelo"

In [ ]:
#Método para cuando clickan en el boton de "Crear Modelo"

def botonCreacion(ventanaRaiz):
        ocultarVent(raiz) #Ocultamos la raiz
        destruirVent(ventanaRaiz) #Destruimos la ventana anterior
        
        
        #Creamos la nueva ventana y la configuramos
        ventanaCreacion=Toplevel() #Creamos una nueva ventana
        ventanaCreacion.iconbitmap("imagen.ico") #Ponemos una imagen a nuestro gusto para la ventana
        ventanaCreacion.resizable(False,False) #Hacemos que la ventana no pueda variar su tamaño.
        
        ventanaCreacion.geometry("650x400") #Tamaño: 650 de ancho y 500 de alto
        barraMenu(ventanaCreacion) #Mostramos el menu
       
        #Creamos el encabezado de la ventana y lo colocamos
        vent=Label(ventanaCreacion,text="Selección de datos", font=("arial bold", 18))
        vent.pack(pady=25)
        
        #creamos buttonCarga y lo configuramos a nuestro gusto
        buttonCarga = Button(ventanaCreacion, text = "Carga de datos", bg = "green",fg = "white",font = arial, command=lambda:ventanaPostCarga(ventanaCreacion)) 
        buttonCarga.config(width = 35, height = 5,anchor=CENTER)
        buttonCarga.pack(pady=20)
    
        #creamos button de Inicio y lo configuramos a nuestro gusto
        buttonInicio = Button(ventanaCreacion, text = "Inicio", bg = "green",fg = "white",font = arial, command=lambda:ventanaInicio(ventanaCreacion))
        buttonInicio.config(anchor=CENTER)
        buttonInicio.place(x=560,y=330)


## Ventana de solicitar Riesgo al pulsar el botón "Solicitar Riesgo"

In [ ]:
#Método para cuando pulsemos solicitar riesgo. Mostramos el ultimo modelo cargado y sus features

def ventPreFormulario(ventanaRaiz):
    
    #Ocultamos la raiz y destruimos la ventana anterior
    ocultarVent(raiz)
    destruirVent(ventanaRaiz)
    
    #Creamos la nueva ventana y la configuramos a nuestro gusto
    ventanaRiesgo=Toplevel() #Nueva ventana 
    ventanaRiesgo.iconbitmap("imagen.ico") #Ponemos una imagen a nuestro gusto para la ventana
    ventanaRiesgo.resizable(False,False) #Hacemos que la ventana no pueda variar su tamaño.
    ventanaRiesgo.geometry("650x400") #Damos 650 de ancho y 500 de alto
    barraMenu(ventanaRiesgo)
    
    #Tratamos con los archivos para saber cual es el ultimo modelo cargado
    rutaFichero=os.getcwd() #Sacamos la ruta donde estamos ejecutando el script
    fileName="LOG_HISTORICO_CARGADOS.txt"
    completeName=rutaFichero + "/" + fileName #Ruta y nombre del archivo donde vamos a escribir
    
    #Obtenemos el ultimo modelo cargado y nos hacemos con su lista de features
    listaUltimasLineas=cargarLogCargados(completeName) 
    lineaCaracteristicas=listaUltimasLineas[0]
    ultimoModeloCargado =rutaFichero + "/" + lineaCaracteristicas
    listaParametrosElegidos=cargarArchivoListaParametrosElegidos(ultimoModeloCargado)
    
    #Configuramos la ventana para mostrar el nombre del ultimo modelo y los features del mismo.
    
    #Creamos el encabezado de la pagina
    
    nombreModelo=lineaCaracteristicas[:-20]
    ultimoModeloLabel=Label(ventanaRiesgo,text="El ultimo Modelo cargado es: " + nombreModelo, font=("arial bold", 12), anchor=CENTER,pady=29)
    ultimoModeloLabel.place(x=0,y=0)
    
    #Creamos el encabezado de la ventana 
    labelHaEscogido=Label(ventanaRiesgo,text="Las características para su modelo son: " , font=("arial bold", 12), anchor=W,pady=15)
    labelHaEscogido.place(x=0, y=50)
    
    Label(ventanaRiesgo,text="", font=("arial bold", 1)).grid(row=0,column=0,padx=5,pady=50)
    Label(ventanaRiesgo,text="", font=("arial bold", 1)).grid(row=1,column=0,padx=5)
    
    
    contCol=0
    contFil=3

    for i in range(len(listaParametrosElegidos)-4):
        if(i!=0):
            contCol+=1
        if(contCol==4):
            contCol=0
            contFil+=1
        
        labeli = Label(ventanaRiesgo,text=str(i+1) + "-" + listaParametrosElegidos[i]).grid(row=contFil,column=contCol,padx=10, pady=5, sticky=W)

    #Para insertar las características de las redes (Capas y Nodos)
    if(contCol+1==4):
        contCol=0
        contFil+=1
    else:
        contCol+=1
    
    labelRed = Label(ventanaRiesgo,text="** NºCapas(RN): " + listaParametrosElegidos[i+1]).grid(row=contFil,column=contCol,padx=10, pady=5, sticky=W)
    
    if(contCol+1==4):
        contCol=0
        contFil+=1
    else:
        contCol+=1
    
    labelRed = Label(ventanaRiesgo,text="** NºNodos(RN): " + listaParametrosElegidos[i+2]).grid(row=contFil,column=contCol,padx=10, pady=5, sticky=W)
            
    #creamos button de aceptar y lo configuramos a nuestro gusto
    buttonNextPage = Button(ventanaRiesgo, text = "Rellenar formulario", bg = "green",fg = "white",font = arial, command=lambda:ventFormulario(ventanaRiesgo))
    buttonNextPage.config(anchor=CENTER)
    buttonNextPage.place(x=430,y=330)
    
    #creamos button de Inicio y lo configuramos a nuestro gusto
    buttonInicio = Button(ventanaRiesgo, text = "Cargar otro modelo", bg = "green",fg = "white",font = arial, command=lambda:ventanaInicio(ventanaRiesgo))
    buttonInicio.config(anchor=CENTER)
    buttonInicio.place(x=10,y=330)
    

## Ventana donde rellenamos el formulario de un nuevo Cliente

In [ ]:
#Ventana donde mostramos la primera página del formulario que debe rellenar un nuevo cliente.

def ventFormulario(ventanaRiesgo):
    #Ocultamos la raiz y destruimos la ventana anterior
    ocultarVent(raiz)
    destruirVent(ventanaRiesgo)
    
    #Creamos la nueva ventana y la configuramos a nuestro gusto
    ventanaRiesgo=Toplevel() #Nueva ventana 
    ventanaRiesgo.iconbitmap("imagen.ico") #Ponemos una imagen a nuestro gusto para la ventana
    ventanaRiesgo.resizable(False,False) #Hacemos que la ventana no pueda variar su tamaño.
    ventanaRiesgo.geometry("650x400") #Damos 650 de ancho y 500 de alto
    barraMenu(ventanaRiesgo)
    
    #Llamamos a la funcion que rellena la primera parte del formulario
    listaAtributos=[]
    listaAtributos=crearFormulario(ventanaRiesgo)
    
    #creamos button de aceptar y lo configuramos a nuestro gusto
    buttonNextPage = Button(ventanaRiesgo, text = "Siguiente", bg = "green",fg = "white",font = arial, command=lambda:ventanaFormularioDos(ventanaRiesgo,listaAtributos))
    buttonNextPage.config(anchor=CENTER)
    buttonNextPage.place(x=520,y=330)

## Función para filtrar los features elegidos por el usuario

In [ ]:
#Método donde filtramos cuales han sido los feature que ha elegido el nuevo usuario.

def mostrarParametrosElegidos(listaParametros):
        
    listaParametrosElegidos=[]
    #print(listaParametros[len(listaParametros)-2].get())
    #print(listaParametros[len(listaParametros)-1].get())
    
    #Si escogemos "Todos" cogemos todos los features
    if(listaParametros[len(listaParametros)-3].get()==1): 
        listaParametrosElegidos.append("ASinSiniestro")
        listaParametrosElegidos.append("Email")
        listaParametrosElegidos.append("FechaCarnet")
        listaParametrosElegidos.append("FechaNacimiento")
        listaParametrosElegidos.append("TipoCarnet")
        listaParametrosElegidos.append("CPConduccion")
        listaParametrosElegidos.append("Cilindrada")
        listaParametrosElegidos.append("ConductorUnico")
        listaParametrosElegidos.append("Marca")
        listaParametrosElegidos.append("Modelo")
        listaParametrosElegidos.append("VehiculoHistorico")
        listaParametrosElegidos.append("FechaMatriculacion")
        listaParametrosElegidos.append("AccidentesConductor")
        listaParametrosElegidos.append("Importe")
        listaParametrosElegidos.append("ModalidadSeleccionada")
        listaParametrosElegidos.append("ClaseVehiculo")
        listaParametrosElegidos.append("Tipo de Pago")
        #Añadimos las caracteristicas de la red neuronal
        #listaParametrosElegidos.append(listaParametros[len(listaParametros)-2])
        #listaParametrosElegidos.append(listaParametros[len(listaParametros)-1])
        
        if(listaParametros[len(listaParametros)-2].get() == ""):
            listaParametrosElegidos.append("3")
        else:
            listaParametrosElegidos.append(listaParametros[len(listaParametros)-2].get())
        
        if(listaParametros[len(listaParametros)-1].get() == ""):
            listaParametrosElegidos.append("100")
        else:
            listaParametrosElegidos.append(listaParametros[len(listaParametros)-1].get())
        
        
        
    #Vamos viendo cuales features han sido los escogidos
    else:
        
        for i in range(len(listaParametros)):
            if(listaParametros[i].get()==1 and i==0):
                listaParametrosElegidos.append("ASinSiniestro")

            elif(listaParametros[i].get()==1 and i==1):
                listaParametrosElegidos.append("Email")

            elif(listaParametros[i].get()==1 and i==2):
                listaParametrosElegidos.append("FechaCarnet")

            elif(listaParametros[i].get()==1 and i==3):
                listaParametrosElegidos.append("FechaNacimiento")

            elif(listaParametros[i].get()==1 and i==4):
                listaParametrosElegidos.append("TipoCarnet")

            elif(listaParametros[i].get()==1 and i==5):
                listaParametrosElegidos.append("CPConduccion")

            elif(listaParametros[i].get()==1 and i==6):
                listaParametrosElegidos.append("Cilindrada")

            elif(listaParametros[i].get()==1 and i==7):
                listaParametrosElegidos.append("ConductorUnico")

            elif(listaParametros[i].get()==1 and i==8):
                listaParametrosElegidos.append("Marca")

            elif(listaParametros[i].get()==1 and i==9):
                listaParametrosElegidos.append("Modelo")

            elif(listaParametros[i].get()==1 and i==10):
                listaParametrosElegidos.append("VehiculoHistorico")

            elif(listaParametros[i].get()==1 and i==11):
                listaParametrosElegidos.append("FechaMatriculacion")

            elif(listaParametros[i].get()==1 and i==12):
                listaParametrosElegidos.append("AccidentesConductor")

            elif(listaParametros[i].get()==1 and i==13):
                listaParametrosElegidos.append("Importe")

            elif(listaParametros[i].get()==1 and i==14):
                listaParametrosElegidos.append("ModalidadSeleccionada")

            elif(listaParametros[i].get()==1 and i==15):
                listaParametrosElegidos.append("ClaseVehiculo")

            elif(listaParametros[i].get()==1 and i==16):
                listaParametrosElegidos.append("Tipo de Pago")
                
            elif(i==18 or i== 19):
                if(listaParametros[i].get()==""):
                    listaParametrosElegidos.append("3")
                else:
                    listaParametrosElegidos.append(listaParametros[i]) 
                
            elif(i==19):
                if(listaParametros[i].get()==""):
                    listaParametrosElegidos.append("100")
                else:
                    listaParametrosElegidos.append(listaParametros[i]) 
                
                
    return listaParametrosElegidos

## Ventana que muestra los resultados obtenidos al cargar un modelo

In [ ]:
#Ventana que muestra los resultados tras cargar un modelo

def ventanaResultadosModeloCargar(ventanaPostParametros, listaParametrosElegidos):
    
    #Ocultamos la raiz
    ocultarVent(raiz)
    
    #Creamos la nueva ventana y la configuramos
    ventanaResultadosModelo=Toplevel() #Creamos una nueva ventana donde mostraremos lso resultados
    ventanaResultadosModelo.iconbitmap("imagen.ico") #Ponemos una imagen a nuestro gusto para la ventana
    ventanaResultadosModelo.geometry("750x500") #Damos 650 de ancho y 500 de alto
    barraMenu(ventanaResultadosModelo)
    
    #Creamos el encabezado de la ventana y la colocamos
    ventanaRes=Label(ventanaResultadosModelo,text="Resultados del modelo cargado", font=("arial bold", 20), anchor=CENTER)
    ventanaRes.pack()
    
    #Mostraremos los resultados obtenidos en cada uno de los clasificadores
    #OBTENEMOS LOS VALORES DE LOS MODELOS
    
    print(listaParametrosElegidos)
    
    resultadoRN=listaParametrosElegidos[len(listaParametrosElegidos)-2]
    resultadoRN=resultadoRN[14:]
    resultadoRF=listaParametrosElegidos[len(listaParametrosElegidos)-1]
    resultadoRF=resultadoRF[14:]
    
    #Mostraremos los resultados obtenidos en cada uno de los clasificadores
        
    #Creamos el encabezado de la ventana y lo colocamos
    ventanaConse=Label(ventanaResultadosModelo,text="(Indica el porcentaje de acierto de dicho modelo)", font=("arial bold", 10), anchor=CENTER)
    ventanaConse.pack()
    
    ventanaConse=Label(ventanaResultadosModelo,text="", font=("arial bold", 10), anchor=CENTER, pady=30)
    ventanaConse.pack()
    
    #Mostraremos los resultados obtenidos en cada uno de los clasificadores
    
    #Redes neuronales
    redNeuronalLabel=Label(ventanaResultadosModelo,    text=" - Red Neuronal: " + resultadoRN, font=("arial bold", 16), anchor=CENTER,pady=10)
    redNeuronalLabel.pack()
    
    #Random Forest
    randomForestLabel=Label(ventanaResultadosModelo,    text="  - Random Forest: " + resultadoRF, font=("arial bold", 16), anchor=CENTER,pady=10)
    randomForestLabel.pack()
    
    #creamos button de Inicio y lo configuramos a nuestro gusto
    buttonInicio = Button(ventanaResultadosModelo, text = "Inicio", bg = "green",fg = "white",font = arial, command=lambda:ventanaInicio(ventanaResultadosModelo))
    buttonInicio.config(anchor=CENTER)
    buttonInicio.place(x=660,y=430)

## Ventana que muestra los resultados obtenidos en base al modelo creado por el usuario

In [ ]:
#Ventana donde mostramos los resultados obtenidos en el modelo

def ventanaResultadosModelo(ventanaPostParametros, listaParametrosElegidos, clean_data,num_nodos,num_capas):

    #Ocultamos la raiz y destruimos la ventana anterior
    destruirVent(ventanaPostParametros)
    ocultarVent(raiz)
    
    #Modelos
    mlp = rn.crearModelo(clean_data,num_nodos,num_capas)
    randomForest = rf.crearModelo(clean_data)
    
    #Resultados de los modelos
    
    resultadosRedNeuronal = str(rn.accuracy(clean_data,num_nodos,num_capas))
    resultadosRandomForest = str(rf.accuracy(clean_data))
    
    #Creamos la nueva ventana y la configuramos a nuestro gusto
    ventanaResultadosModelo=Toplevel()
    ventanaResultadosModelo.iconbitmap("imagen.ico") #Ponemos una imagen a nuestro gusto para la ventana
    ventanaResultadosModelo.geometry("750x500") #Damos 650 de ancho y 500 de alto
    barraMenu(ventanaResultadosModelo)
    
    #Creamos el encabezado de la ventana y lo colocamos
    ventanaRes=Label(ventanaResultadosModelo,text="Resultados del modelo creado", font=("arial bold", 20), anchor=CENTER)
    ventanaRes.pack()
    
    #Creamos el encabezado de la ventana y lo colocamos
    ventanaConse=Label(ventanaResultadosModelo,text="(Indica el porcentaje de acierto de dicho modelo)", font=("arial bold", 10), anchor=CENTER)
    ventanaConse.pack()
    
    ventanaConse=Label(ventanaResultadosModelo,text="", font=("arial bold", 10), anchor=CENTER, pady=30)
    ventanaConse.pack()
    
    #Mostraremos los resultados obtenidos en cada uno de los clasificadores
    
    #Redes neuronales
    redNeuronalLabel=Label(ventanaResultadosModelo,    text=" - Red Neuronal: " + resultadosRedNeuronal + "%", font=("arial bold", 16), anchor=CENTER,pady=10)
    redNeuronalLabel.pack()
    
    #Random Forest
    randomForestLabel=Label(ventanaResultadosModelo,    text="  - Random Forest: " + resultadosRandomForest + "%", font=("arial bold", 16), anchor=CENTER,pady=10)
    randomForestLabel.pack()
    
    #creamos button de Inicio y lo configuramos a nuestro gusto
    buttonInicio = Button(ventanaResultadosModelo, text = "Inicio", bg = "green",fg = "white",font = arial, command=lambda:ventanaInicio(ventanaResultadosModelo))
    buttonInicio.config(anchor=CENTER)
    buttonInicio.place(x=660,y=430)
    
    #Creamos un button para guardar el modelo creado. Tenemos que guardar los features elegidos: listaParametrosElegidos
    buttonInicio = Button(ventanaResultadosModelo, text = "Guardar Modelo", bg = "green",fg = "white",font = arial, command=(lambda:guardarModeloEmerg(listaParametrosElegidos, mlp, randomForest, resultadosRedNeuronal, resultadosRandomForest)))
    buttonInicio.config(anchor=CENTER)
    buttonInicio.place(x=10,y=430)

## Ventana Post Parámetros

In [ ]:
#Ventana donde Mostramos los features elegidos para generar el modelo

def ventanaPostParametros(ventanaParametros,listaParametros,ruta_fichero):
    
    #Ocultamos la raiz y destruimos la ventana anterior
    destruirVent(ventanaParametros)
    ocultarVent(raiz)
    
    #Creamos la nueva ventana y la configuramos a nuestro gusto
    ventanaPostParametros=Toplevel() #Creamos una nueva ventana donde mostraremos lso resultados
    ventanaPostParametros.resizable(False,False) #Hacemos que la ventana no pueda variar su tamaño.
    ventanaPostParametros.iconbitmap("imagen.ico") #Ponemos una imagen a nuestro gusto para la ventana
    ventanaPostParametros.geometry("650x400") #Damos 650 de ancho y 500 de alto
    barraMenu(ventanaPostParametros)
    
    
    #Configuramos la ventana para mostrar el titulo y los features elegidos
    
    #Creamos el encabezado de la ventana y la colcamos
    labelHaEscogido=Label(ventanaPostParametros,text="Las características para su modelo son: " , font=("arial bold", 18), anchor=W,pady=29).place(x=0,y=0)
    Label(ventanaPostParametros, font=("arial bold", 10), pady=29).grid(row=0,column=5,padx=10, pady=5)

    #Funcion auxilar para mostrar solo aquellos parametros que han sido elegidos.
    listaParametrosElegidos=[]
    listaParametrosElegidos=mostrarParametrosElegidos(listaParametros)
    
    contCol=0
    contFil=2
    
    #Traspaso y tratamiento de los datos en base a los features elegidos
    data = fn.traspasoDatos(ruta_fichero)
    clean_data = fn.limpiezaDatos(data,listaParametrosElegidos)
    
    #print(listaParametrosElegidos)
    
    for i in range(len(listaParametrosElegidos)-2):
        if(i!=0):
            contCol+=1
        if(contCol==4):
            contCol=0
            contFil+=1
        
        labeli = Label(ventanaPostParametros,text=str(i+1) + "-" + listaParametrosElegidos[i]).grid(row=contFil,column=contCol,padx=10, pady=5, sticky=W)
        
    num_capas= listaParametrosElegidos[i+1]
    num_nodos= listaParametrosElegidos[i+2]
    
    if(contCol+1==4):
        contCol=0
        contFil+=1
    else:
        contCol+=1
    
    labelRed = Label(ventanaPostParametros,text="** NºCapas(RN): " + num_capas).grid(row=contFil,column=contCol,padx=10, pady=5, sticky=W)
    
    if(contCol+1==4):
        contCol=0
        contFil+=1
    else:
        contCol+=1
    
    
    
    labelRed = Label(ventanaPostParametros,text="** NºNodos(RN): " + num_nodos).grid(row=contFil,column=contCol,padx=10, pady=5, sticky=W)

    labelHaEscogido=Label(ventanaPostParametros,text="Si desea cambiar algun feature debe volver a cargar un nuevo fichero." , font=("arial bold", 13), anchor=W,pady=29)
    labelHaEscogido.place(x=0,y=250)
    
    #creamos button de Crear Modelo y lo configuramos a nuestro gusto
    buttonCrearModelo = Button(ventanaPostParametros, text = "Crear Modelo", bg = "green",fg = "white",font = arial, command=lambda:ventanaResultadosModelo(ventanaPostParametros, listaParametrosElegidos,clean_data,num_nodos,num_capas))
    buttonCrearModelo.config(anchor=CENTER)
    buttonCrearModelo.place(x=480,y=330)
    
    #creamos button de Inicio y lo configuramos a nuestro gusto
    buttonRetroceder = Button(ventanaPostParametros, text = "Cargar nuevos Datos", bg = "green",fg = "white",font = arial, command=lambda:ventanaPostCarga(ventanaPostParametros))
    buttonRetroceder.config(anchor=CENTER)
    buttonRetroceder.place(x=10,y=330)

## Ventana para elegir las caracteristicas propias de la red neuronal

In [ ]:
def ventanaParamRedNeuronal(ventanaParametros,listaParametros,ruta_fichero):

    #Ocultamos la raiz y destruimos la ventana anterior
    destruirVent(ventanaParametros) #Destruimos la ventana anterior
    ocultarVent(raiz)
    
    #Creamos la nueva ventana y la configuramos a nuestro gusto.
    ventanaParametrosRedNeuronal=Toplevel() #Creamos una nueva ventana donde mostraremos lso resultados
    ventanaParametrosRedNeuronal.resizable(False,False) #Hacemos que la ventana no pueda variar su tamaño.
    ventanaParametrosRedNeuronal.iconbitmap("imagen.ico") #Ponemos una imagen a nuestro gusto para la ventana
    ventanaParametrosRedNeuronal.geometry("650x400") #Damos 650 de ancho y 500 de alto
    barraMenu(ventanaParametrosRedNeuronal)
    
    #Creamos el encabezado de la pagina
    parametrosLabel=Label(ventanaParametrosRedNeuronal,text="Elige los parámetros de su red neuronal", font=("arial bold", 18), pady=29).place(x=0,y=0)
    Label(ventanaParametrosRedNeuronal,text="", font=("arial bold", 10), pady=29).grid(row=0,column=10,padx=10, pady=5)
    Label(ventanaParametrosRedNeuronal,text="", font=("arial bold", 10), pady=29).grid(row=3,column=4,padx=10, pady=5)

    #Capas
    var_capas1 = StringVar()
    cuadroNombre=Entry(ventanaParametrosRedNeuronal, textvariable=var_capas1)
    cuadroNombre.grid(row=2,column=1, padx=5, pady=10)
    cuadroNombre.config(fg="red",justify="center")
    
    capasLabel=Label(ventanaParametrosRedNeuronal, text="Nº Capas: ")
    capasLabel.grid(row=2,column=0, padx=5, pady=10)
    
    #Capas
    var_nodos = StringVar()
    cuadroNombre=Entry(ventanaParametrosRedNeuronal, textvariable=var_nodos)
    cuadroNombre.grid(row=2,column=3, padx=5, pady=10)
    cuadroNombre.config(fg="red",justify="center")
    
    capasLabel=Label(ventanaParametrosRedNeuronal, text="Nº Nodos por capa: ")
    capasLabel.grid(row=2,column=2, padx=5, pady=10)
        
    listaParametros.append(var_capas1)
    listaParametros.append(var_nodos)
    
    Label(ventanaParametrosRedNeuronal,text="**Configuración por defecto: 3 Capas y 100 Nodos por capa", font=("arial bold", 10), pady=29).place(x=0,y=125)

    
    
    #creamos button de Inicio y lo configuramos a nuestro gusto
    buttonAceptar = Button(ventanaParametrosRedNeuronal, text = "Aceptar", bg = "green",fg = "white",font = arial, command=lambda:ventanaPostParametros(ventanaParametrosRedNeuronal,listaParametros,ruta_fichero))
    buttonAceptar.config(anchor=CENTER)
    buttonAceptar.place(x=540,y=330)
    

## Ventana para Elegir el excell para crear el nuevo modelo

In [ ]:
#En esta ventana cargaremos el excell de datos y mostraremos los posibles features a la hora de crear un nuevo modelo

def ventanaPostCarga(ventanaCreacion):
    listaParametros=[]
    
    #en ruta_fichero obtendremos la ruta del excell seleccionado.
    ruta_fichero=cargaFicheroExcell()
    
    #Ocultamos la raiz y destruimos la ventana anterior
    destruirVent(ventanaCreacion) #Destruimos la ventana anterior
    ocultarVent(raiz)
    
    #Creamos la nueva ventana y la configuramos a nuestro gusto.
    ventanaParametros=Toplevel() #Creamos una nueva ventana donde mostraremos lso resultados
    ventanaParametros.resizable(False,False) #Hacemos que la ventana no pueda variar su tamaño.
    ventanaParametros.iconbitmap("imagen.ico") #Ponemos una imagen a nuestro gusto para la ventana
    ventanaParametros.geometry("650x400") #Damos 650 de ancho y 500 de alto
    barraMenu(ventanaParametros)
    
    #Obtenemos la lista de parametros obtenidos
    listaParametros=muestraParametros(ventanaParametros,listaParametros)
    
    #creamos button de Inicio y lo configuramos a nuestro gusto
    buttonAceptar = Button(ventanaParametros, text = "Aceptar", bg = "green",fg = "white",font = arial, command=lambda:ventanaParamRedNeuronal(ventanaParametros,listaParametros,ruta_fichero))
    buttonAceptar.config(anchor=CENTER)
    buttonAceptar.place(x=540,y=330)


## Muestra de parametros en checkButtons

In [ ]:
#Método donde mostraremos los posibles features y el usuario eligirá con cuales quiere crear el nuevo modelo.

def muestraParametros(ventanaParametros, listaParametros):    
    
    #Creamos el encabezado de la pagina
    parametrosLabel=Label(ventanaParametros,text="Seleccione las características de su modelo ", font=("arial bold", 18), pady=29).place(x=0,y=0)
    Label(ventanaParametros,text="", font=("arial bold", 10), pady=29).grid(row=0,column=10,padx=10, pady=5)
    
    #Mostramos los checkbuttons de los posibles features
    
    var_año_nacimiento=IntVar()
    Checkbutton(ventanaParametros, text="Año_Nacimiento", variable=var_año_nacimiento, onvalue=1, offvalue=0).grid(row=2,column=0,sticky=W,padx=20, pady=5)
    
    var_Año_carnet=IntVar()
    Checkbutton(ventanaParametros, text="Año_carnet",variable=var_Año_carnet, onvalue=1, offvalue=0).grid(row=3,column=0,sticky=W,padx=20, pady=5)
    
    var_email=IntVar()
    Checkbutton(ventanaParametros, text="Email",variable=var_email, onvalue=1, offvalue=0).grid(row=4,column=0,sticky=W,padx=20, pady=5)

    var_condUnico=IntVar()
    Checkbutton(ventanaParametros, text="CondUnico",variable=var_condUnico, onvalue=1, offvalue=0).grid(row=2,column=1,sticky=W,padx=20, pady=5)
   
    var_tipoCarnet=IntVar()
    Checkbutton(ventanaParametros, text="Tipo_Carnet",variable=var_tipoCarnet, onvalue=1, offvalue=0).grid(row=3,column=1,sticky=W,padx=20, pady=5)
        
    var_marca=IntVar()
    Checkbutton(ventanaParametros, text="Marca",variable=var_marca, onvalue=1, offvalue=0).grid(row=5,column=1,sticky=W,padx=20, pady=5)
    
    var_modelo=IntVar()
    Checkbutton(ventanaParametros, text="Modelo",variable=var_modelo, onvalue=1, offvalue=0).grid(row=6,column=1,sticky=W,padx=20, pady=5)
   
    var_cilindrada=IntVar()
    Checkbutton(ventanaParametros, text="Cilindrada",variable=var_cilindrada, onvalue=1, offvalue=0).grid(row=2,column=2,sticky=W,padx=20, pady=5)  
    
    var_clase_vehiculo=IntVar()
    Checkbutton(ventanaParametros, text="Clase_vehiculo",variable=var_clase_vehiculo, onvalue=1, offvalue=0).grid(row=3,column=2,sticky=W,padx=20, pady=5)
    
    var_año_matriculacion=IntVar()
    Checkbutton(ventanaParametros, text="Año_matriculación",variable=var_año_matriculacion, onvalue=1, offvalue=0).grid(row=4,column=2,sticky=W,padx=20, pady=5)
    
    var_vehiculo_historico=IntVar()
    Checkbutton(ventanaParametros, text="Vehiculo_historico",variable=var_vehiculo_historico, onvalue=1, offvalue=0).grid(row=5,column=2,sticky=W,padx=20, pady=5)
    
    var_tipo_pago=IntVar()
    Checkbutton(ventanaParametros, text="Tipo_pago",variable=var_tipo_pago, onvalue=1, offvalue=0).grid(row=6,column=2,sticky=W,padx=20, pady=5)
    
    var_modalidad=IntVar()
    Checkbutton(ventanaParametros, text="Modalidad",variable=var_modalidad, onvalue=1, offvalue=0).grid(row=5,column=0,sticky=W,padx=20, pady=5)
    
    var_cp_conduccion=IntVar()
    Checkbutton(ventanaParametros, text="CP_Conducción",variable=var_cp_conduccion, onvalue=1, offvalue=0).grid(row=6,column=0,sticky=W,padx=20, pady=5)
    
    var_importe=IntVar()
    Checkbutton(ventanaParametros, text="Importe",variable=var_importe, onvalue=1, offvalue=0).grid(row=4,column=1,sticky=W,padx=20, pady=5)
    
    var_a_sin_siniestro=IntVar()
    Checkbutton(ventanaParametros, text="Años_sin_siniestro",variable=var_a_sin_siniestro, onvalue=1, offvalue=0).grid(row=2,column=3,sticky=W,padx=20, pady=5)
    
    var_accidentes_conductor=IntVar()
    Checkbutton(ventanaParametros, text="Accidentes Conductor",variable=var_accidentes_conductor, onvalue=1, offvalue=0).grid(row=3,column=3,sticky=W,padx=20, pady=5)
    
    var_todas=IntVar()
    Checkbutton(ventanaParametros, text="TODAS",variable=var_todas, onvalue=1, offvalue=0).grid(row=4,column=3,sticky=W,padx=20, pady=5)
    
    #Metemos las opciones elegidas a la listaDeParametros.
    #Devuelve 1 si ha sido clickado y 1 si no ha sido clickeado
    listaParametros.append(var_a_sin_siniestro) #0
    listaParametros.append(var_email) #1
    listaParametros.append(var_Año_carnet) #2.
    listaParametros.append(var_año_nacimiento) #3.
    listaParametros.append(var_tipoCarnet) #4.
    listaParametros.append(var_cp_conduccion) #5.
    listaParametros.append(var_cilindrada) #6.
    listaParametros.append(var_condUnico) #7.
    listaParametros.append(var_marca) #8 
    listaParametros.append(var_modelo) #9.
    listaParametros.append(var_vehiculo_historico) #10.
    listaParametros.append(var_año_matriculacion) #11.
    listaParametros.append(var_accidentes_conductor) #12.
    listaParametros.append(var_importe) #13.
    listaParametros.append(var_modalidad) #14.
    listaParametros.append(var_clase_vehiculo) #15.
    listaParametros.append(var_tipo_pago) #16.
    listaParametros.append(var_todas) #17.
    
    return listaParametros

## Función para cargar un modelo RF

In [ ]:
#Metodo que devuelve la ruta del modelo de RF que que queremos abrir
def cargaModeloRF():
    
    fichero=filedialog.askopenfilename(title="Seleccione el modelo Random Forest que desea cargar por favor", initialdir="Escritorio", filetypes=(
                                                                                                                    ("Todos los ficheros", "*.*")
                                                                                                                    ,))
    return fichero

## Función para cargar un modelo RN

In [ ]:
#Metodo que devuelve la ruta del del modelo de RN que queremos abrir
def cargaModeloRN():
    
    fichero=filedialog.askopenfilename(title="Seleccione el modelo Red Neuronal que desea cargar por favor", initialdir="Escritorio", filetypes=(
                                                                                                                    ("Todos los ficheros", "*.*")
                                                                                                                    ,))
    return fichero

## Función Para Cargar .TXT

In [ ]:
#Metodo que devuelve la ruta del fichero TXT que queremos abrir
def cargaFicheroTXT():
    
    fichero=filedialog.askopenfilename(title="Seleccione el documento .TXT que desea cargar por favor", initialdir="Escritorio", filetypes=(
                                                                                                                    ("Fichero de texto", "*.txt")
                                                                                                                    ,))
    return fichero

## Función para Cargar Datos Excell

In [ ]:
#Metodo que devuelve la ruta del fichero EXCELL que queremos abrir
def cargaFicheroExcell():
    
    fichero=filedialog.askopenfilename(title="Seleccione el documento excell donde tiene los datos a cargar por favor", initialdir="Escritorio", filetypes=(
                                                                                                                    ("Ficheros de Excel", "*.xlsx")
                                                                                                                    ,))
    return fichero

## CrearFormulario (Primera Página)

In [ ]:
#Metodo donde se implementa el formulario a rellenar para un nuevo cliente

def crearFormulario(ventanaRiesgo):
    ocultarVent(raiz)
    
    #Creamos el encabezado de la pagina
    formularioClienteLabel=Label(ventanaRiesgo,text="Formulario Cliente", font=("arial bold", 18), anchor=CENTER,pady=29)
    formularioClienteLabel.grid(row=0,column=3)
    
    #CUADROS a rellenar
    
    #NOMBRE
    var_nombre = StringVar()
    cuadroNombre=Entry(ventanaRiesgo, textvariable=var_nombre)
    cuadroNombre.grid(row=2,column=4, padx=5, pady=10)
    cuadroNombre.config(fg="red",justify="center")
    
    #AÑO DE NACIMIENTO
    var_año_nacimiento = StringVar()
    cuadroAñoNacimiento=Entry(ventanaRiesgo, textvariable=var_año_nacimiento)
    cuadroAñoNacimiento.grid(row=3,column=4, padx=5)
    cuadroAñoNacimiento.config(fg="red",justify="center")
    
    #APELLIDO
    var_apellido = StringVar()
    cuadroApellido=Entry(ventanaRiesgo, textvariable=var_apellido)
    cuadroApellido.grid(row=2,column=7, padx=5)
    cuadroApellido.config(fg="red",justify="center")
    
    #AÑO CARNET DE CONDUCIR
    var_año_carnet = StringVar()
    cuadroAñoCarnetConducir=Entry(ventanaRiesgo, textvariable=var_año_carnet)
    cuadroAñoCarnetConducir.grid(row=3,column=7, padx=5, pady=10)
    cuadroAñoCarnetConducir.config(fg="red",justify="center")
    
    #EMAIL
    var_email = StringVar()
    cuadroEmail=Entry(ventanaRiesgo, textvariable=var_email)
    cuadroEmail.grid(row=4,column=4, padx=5)
    cuadroEmail.config(fg="red",justify="center")
    
    #años_sin_sinietro
    var_años_sin_siniestro = StringVar()
    cuadroAños_sin_siniestro=Entry(ventanaRiesgo, textvariable=var_años_sin_siniestro)
    cuadroAños_sin_siniestro.grid(row=5,column=7, padx=5)
    cuadroAños_sin_siniestro.config(fg="red",justify="center")
    
    #CP_Conduccion
    var_CP_Conduccion = StringVar()
    cuadroCPConduccion=Entry(ventanaRiesgo, textvariable=var_CP_Conduccion)
    cuadroCPConduccion.grid(row=5,column=4, padx=5, pady=10)
    cuadroCPConduccion.config(fg="red",justify="center")
     
    #ETIQUETAS de los cuadros
    nombreLabel=Label(ventanaRiesgo, text="Nombre: ")
    nombreLabel.grid(row=2,column=3, padx=5, pady=10)
    
    nacimientoLabel=Label(ventanaRiesgo, text="Año Nacimiento: ")
    nacimientoLabel.grid(row=3,column=3, padx=5, pady=10)
    
    apellidoLabel=Label(ventanaRiesgo, text="Apellido: ")
    apellidoLabel.grid(row=2,column=6, padx=5, pady=10)
    
    añoCarnetLabel=Label(ventanaRiesgo, text="Año Carnet: ")
    añoCarnetLabel.grid(row=3,column=6, padx=5, pady=10)
    
    emailLabel=Label(ventanaRiesgo, text="E-mail: ")
    emailLabel.grid(row=4,column=3,padx=5, pady=10)
    
    años_sin_siniestroLabel=Label(ventanaRiesgo, text="Años sin siniestro: ")
    años_sin_siniestroLabel.grid(row=5,column=6,padx=5, pady=10)
    
    CPConduccionLabel=Label(ventanaRiesgo, text="CP Conducción: ")
    CPConduccionLabel.grid(row=5,column=3,padx=5, pady=10)
    
    cilindradaLabel=Label(ventanaRiesgo, text="Cilindrada: ")
    cilindradaLabel.grid(row=6,column=6,padx=5, pady=10)
    
    #GENERO DEL CLIENTE
    generoLabel=Label(ventanaRiesgo, text="Género: ") #Etiqueta con el sexo del cliente
    generoLabel.grid(row=4,column=6, pady=10)  
    
    var_genero=StringVar()
    comboGenero = ttk.Combobox(ventanaRiesgo, width=17, state="readonly", textvariable=var_genero)
    comboGenero["values"] = ["Masculino","Femenino"]
    #comboGenero.place(x=470,y=185)
    comboGenero.grid(row=4,column=7,padx=5, pady=10)
    
    #Cilindrada
    var_Cilindrada=StringVar()
    cuadroCilindrada=Entry(ventanaRiesgo, textvariable=var_Cilindrada)
    cuadroCilindrada.grid(row=6,column=7,padx=5, pady=10)
    cuadroCilindrada.config(fg="red",justify="center")
        
    #TIPO_CARNET
    generoLabel=Label(ventanaRiesgo, text="Tipo de carnet: ") #Etiqueta con el sexo del cliente
    generoLabel.grid(row=6,column=3, pady=10)  
    
    var_tipoCarnet=StringVar()
    comboTipoCarnet = ttk.Combobox(ventanaRiesgo, width=17, state="readonly", textvariable=var_tipoCarnet)
    comboTipoCarnet["values"] = ["AM","A1", "A2", "A", "B", "BE", "C1", "C1E", "C", "CE", "D1", "D1E", "D", "DE"]
    comboTipoCarnet.grid(row=6,column=4, padx=5, pady=10)
    
    #Lista de atributos de un nuevo cliente.
    
    listaAtributos=[]
    listaAtributos.append(var_nombre) #Añadimos el atributo nombre a la lista
    listaAtributos.append(var_apellido) #Añadimos el atributo del apellido a la lista
    listaAtributos.append(var_genero) #Añadimos el atributo del genero a la lista
    listaAtributos.append(var_años_sin_siniestro) #Añadimos el atributo años_sin_siniestro a la lista
    listaAtributos.append(var_email) #Añadimos el atributo del mail a la lista
    listaAtributos.append(var_año_carnet) #Añadimos el atributo del año de carnet de conducir a la lista    
    listaAtributos.append(var_año_nacimiento) #Añadimos el atributo año de nacimiento a la lista
    listaAtributos.append(var_tipoCarnet) #Añadimos el atributo del tipoCarnet a la lista
    listaAtributos.append(var_CP_Conduccion) #Añadimos el atributo de CP_Conduccion a la lista
    listaAtributos.append(var_Cilindrada) #Añadimos el atributo de cilindrada a la lista

    
    return listaAtributos

## CrearFormulario II (Segunda Página)

In [ ]:
#Metodo donde se implementa la segunda pagina del formulario a rellenar para un nuevo cliente

def crearFormularioDos(ventanaRiesgo2, listaAtributos):
    ocultarVent(raiz)
    
    #CUADROS a rellenar
    
    condUnicoLabel=Label(ventanaRiesgo2, text="¿Conductor único?: ")
    condUnicoLabel.grid(row=6,column=6,padx=5, pady=10)

    #CONDUCTOR UNICO
    var_condUnico=StringVar()
    comboConductor = ttk.Combobox(ventanaRiesgo2, width=17, state="readonly", textvariable=var_condUnico)
    comboConductor["values"] = ["SI","NO"]
    comboConductor.grid(row=6,column=7,padx=5)
    
    
    
    #Marca del coche
    var_marca = StringVar()
    cuadroMarca=Entry(ventanaRiesgo2, textvariable=var_marca)
    cuadroMarca.grid(row=2,column=4, padx=5, pady=10)
    cuadroMarca.config(fg="red",justify="center")
    
    #Modelo del coche
    var_modelo = StringVar()
    cuadroModelo=Entry(ventanaRiesgo2, textvariable=var_modelo)
    cuadroModelo.grid(row=2,column=7, padx=5)
    cuadroModelo.config(fg="red",justify="center")
    
    #Accidentes_conductor
    var_accidientes_conductor = StringVar()
    cuadroAciddentes_conductor=Entry(ventanaRiesgo2, textvariable=var_accidientes_conductor)
    cuadroAciddentes_conductor.grid(row=3,column=4, padx=5)
    cuadroAciddentes_conductor.config(fg="red",justify="center")
    
    #Clase Vehiculo
    var_claseVehiculo = StringVar()
    cuadroClaseVehiculo=Entry(ventanaRiesgo2, textvariable=var_claseVehiculo)
    cuadroClaseVehiculo.grid(row=4,column=4, padx=5)
    cuadroClaseVehiculo.config(fg="red",justify="center")
    
    #Año Matriculacion
    var_año_matriculacion = StringVar()
    cuadroAñoMatriculacion=Entry(ventanaRiesgo2, textvariable=var_año_matriculacion)
    cuadroAñoMatriculacion.grid(row=4,column=7, padx=5)
    cuadroAñoMatriculacion.config(fg="red",justify="center")
    
    
    #Importe
    var_importe = StringVar()
    cuadroAñoMatriculacion=Entry(ventanaRiesgo2, textvariable=var_importe)
    cuadroAñoMatriculacion.grid(row=6,column=4, padx=5)
    cuadroAñoMatriculacion.config(fg="red",justify="center")
    
    
    #ETIQUETAS de los cuadros
    marcaLabel=Label(ventanaRiesgo2, text="Marca: ")
    marcaLabel.grid(row=2,column=3, padx=5, pady=10)
    
    modeloLabel=Label(ventanaRiesgo2, text="Modelo: ")
    modeloLabel.grid(row=2,column=6, padx=5, pady=10)
    
    cilindradaLabel=Label(ventanaRiesgo2, text="Nº Accidentes del conductor: ")
    cilindradaLabel.grid(row=3,column=3, padx=5, pady=10)
    
    veh_historico_Label=Label(ventanaRiesgo2, text="¿Vehiculo Histórico?: ")
    veh_historico_Label.grid(row=3,column=6, padx=5, pady=10)
    
    clase_vehiculo_Label=Label(ventanaRiesgo2, text="Clase vehiculo: ")
    clase_vehiculo_Label.grid(row=4,column=3,padx=5, pady=10)
    
    año_matric_Label=Label(ventanaRiesgo2, text="Año matriculación vehiculo: ")
    año_matric_Label.grid(row=4,column=6,padx=5, pady=10)
    
    tipoPagoLabel=Label(ventanaRiesgo2, text="Tipo de Pago: ")
    tipoPagoLabel.grid(row=5,column=3,padx=5, pady=10)
    
    modalidadLabel=Label(ventanaRiesgo2, text="Modalidad: ")
    modalidadLabel.grid(row=5,column=6,padx=5, pady=10)
    
    importeLabel=Label(ventanaRiesgo2, text="Importe: ")
    importeLabel.grid(row=6,column=3,padx=5, pady=10)
    
    #Vehiculo Historico
    var_vehiculo_historico=StringVar()
    comboVehiculoHistorico = ttk.Combobox(ventanaRiesgo2, width=17, state="readonly", textvariable=var_vehiculo_historico)
    comboVehiculoHistorico["values"] = ["SI","NO"]
    comboVehiculoHistorico.grid(row=3,column=7,padx=5, pady=10)
    
    #Tipo de Pago
    var_tipoPago=StringVar()
    comboTipoPago = ttk.Combobox(ventanaRiesgo2, width=17, state="readonly", textvariable=var_tipoPago)
    comboTipoPago["values"] = ["Anual","Semestral","Trimestral"]
    comboTipoPago.grid(row=5,column=4, padx=5, pady=10)
    
    #Modalidad
    var_modalidad=StringVar()
    comboModalidad = ttk.Combobox(ventanaRiesgo2, width=17, state="readonly", textvariable=var_modalidad)
    comboModalidad["values"] = ["Basica","Ampliada","Todo Riesgo"]
    comboModalidad.grid(row=5,column=7, padx=5, pady=10)
    
    #Lista de atributos de un nuevo cliente.
    
    listaAtributos.append(var_condUnico) #Añadimos el atributo marca del vehiculo a la lista
    listaAtributos.append(var_marca) #Añadimos el atributo marca del vehiculo a la lista
    listaAtributos.append(var_modelo) #Añadimos el atributo modelo del vehiculo a la lista
    listaAtributos.append(var_vehiculo_historico) #Añadimos el atributo del vehiculo_historico a la lista
    listaAtributos.append(var_año_matriculacion) #Añadimos el atributo del año de matriculacion a la lista
    listaAtributos.append(var_accidientes_conductor) #Añadimos el atributo del numero de accidentes a la lista
    listaAtributos.append(var_tipoPago) #Añadimos el atributo del tipo de pago a la lista
    listaAtributos.append(var_importe) #Añadimos el atributo del importe a la lista
    listaAtributos.append(var_modalidad) #Añadimos el atributo de la modalidad a la lista
    listaAtributos.append(var_claseVehiculo) #Añadimos el atributo de la clase de vehiculo a la lista

    
    return listaAtributos

## Ventana de la segunda parte del formulario

In [ ]:
#Ventana donde obtenemos la segunda parte del formulario del usuario

def ventanaFormularioDos(ventanaRiesgo,listaAtributos):
    #Ocultamos la raiz y destruimos la ventana anterior
    destruirVent(ventanaRiesgo) #Destruimos la ventana anterior
    ocultarVent(raiz)
    
    #Creamos una nueva ventana y la configuramos a nuestro gusto
    ventanaRiesgo2=Toplevel() #Creamos una nueva ventana donde mostraremos lso resultados
    ventanaRiesgo2.iconbitmap("imagen.ico") #Ponemos una imagen a nuestro gusto para la ventana
    ventanaRiesgo2.geometry("670x400") #Damos 650 de ancho y 500 de alto
    barraMenu(ventanaRiesgo2)
    
    #Creamos el encabezado de la pagina
    formularioClienteLabel=Label(ventanaRiesgo2,text="Formulario Cliente", font=("arial bold", 18), anchor=CENTER,pady=29)
    formularioClienteLabel.grid(row=0,column=3)

    listaAtributos=crearFormularioDos(ventanaRiesgo2,listaAtributos)
    
    #creamos button de aceptar y lo configuramos a nuestro gusto
    buttonAceptar = Button(ventanaRiesgo2, text = "Aceptar", bg = "green",fg = "white",font = arial, command=lambda:ventanaPredicciones(ventanaRiesgo2, listaAtributos))
    buttonAceptar.config(anchor=CENTER)
    buttonAceptar.place(x=540,y=330)

## Ventana donde procesamos los datos cargados en el formulario

In [ ]:
#En esta ventan tratamos los datos que nos ha insertado el nuevo cliente en el formulario

def ventanaPredicciones(ventanaRiesgo2, listaAtributos):
    #Ocultamos la raiz y destruimos la ventana anterior
    destruirVent(ventanaRiesgo2) 
    ocultarVent(raiz)
    
#     for i in range(len(listaAtributos)):
#         print(listaAtributos[i].get())
    
    #Creamos la nueva ventana y la configuramos a nuestro gusto
    ventanaCalculacion=Toplevel() #Creamos una nueva ventana donde mostraremos lso resultados
    ventanaCalculacion.iconbitmap("imagen.ico") #Ponemos una imagen a nuestro gusto para la ventana
    ventanaCalculacion.geometry("650x400") #Damos 650 de ancho y 500 de alto
    barraMenu(ventanaCalculacion)
    
    #Creamos el encabezado de la ventana y lo colcamos
    ventCal=Label(ventanaCalculacion,text="SE ESTAN CALCULANDO LAS PREDICCIONES... " , font=("arial bold", 18), anchor=CENTER,pady=29)
    ventCal.pack()
    
    #creamos button de "Ver resultados" y lo configuramos a nuestro gusto
    buttonResultados = Button(ventanaCalculacion, text = "Ver Resultados", bg = "green",fg = "white",font = arial, command=lambda:resultadosPredicciones(ventanaCalculacion, listaAtributos))
    buttonResultados.config(anchor=CENTER)
    buttonResultados.place(x=460,y=330)

## Ventana donde se muestran los resultados de las Predicciones

In [ ]:
#Muestra los resultados en la ventana correspondiente

def resultadosPredicciones(ventanaCalculacion, listaAtributos):
    
    #Tratamos con los archivos para saber cual es el ultimo modelo cargado
    rutaFichero=os.getcwd() #Sacamos la ruta donde estamos ejecutando el script
    fileName="LOG_HISTORICO_CARGADOS.txt"
    completeName=rutaFichero + "/" + fileName #Ruta y nombre del archivo donde vamos a escribir
    
    #Obtenemos el ultimo modelo cargado y nos hacemos con su lista de features
    listaUltimasLineas=cargarLogCargados(completeName) 
    
    ruta_fichero_RF=rutaFichero + "/" + listaUltimasLineas[1] #Ruta y nombre del modelo RF
    ruta_fichero_RN=rutaFichero + "/" + listaUltimasLineas[2] #Ruta y nombre del modelo RN
    
    mlp = cargarModelo(ruta_fichero_RN) #Obtenemos el modelo de la red neuronal
    randomForest = cargarModelo(ruta_fichero_RF) #Obtenemos el modelo del Random Forest
    
    
    #ocultamos la raiz y destruimos la ventana anterior
    destruirVent(ventanaCalculacion) #Destruimos la ventana anterior
    ocultarVent(raiz)
    
    #Creamos la nueva ventana y la configuramos a nuestro gusto
    ventanaResultados=Toplevel() #Creamos una nueva ventana donde mostraremos lso resultados
    ventanaResultados.iconbitmap("imagen.ico") #Ponemos una imagen a nuestro gusto para la ventana
    ventanaResultados.geometry("750x500") #Damos 650 de ancho y 500 de alto
    barraMenu(ventanaResultados)
    
    #Elegimos que prefijo y que nombre vamos a mostrar en la etiqueta de la ventana de predicciones
    if listaAtributos[2].get() == "Femenino":
        prefijo= "Sra. "
    else:
        prefijo= "Sr. "

    nombre=listaAtributos[0].get()
    apellido=listaAtributos[1].get()
    
    #Creamos el encabezado de la ventana y la colocamos
    ventanaRes=Label(ventanaResultados,text="Resultados de " + prefijo + nombre + " " + apellido, font=("arial bold", 18), anchor=CENTER)
    ventanaRes.pack()
    
    #Creamos el encabezado de la ventana y lo colocamos
    ventanaConse=Label(ventanaResultados,text="(Indica el porcentaje predicho en cada uno de los valores)", font=("arial bold", 10), anchor=CENTER)
    ventanaConse.pack()
    
    ventanaConse=Label(ventanaResultados,text="", font=("arial bold", 10), anchor=CENTER, pady=15)
    ventanaConse.pack()
    
    #Mostraremos los resultados obtenidos en cada uno de los clasificadores
    
    dataSolicitado = {'ASinSiniestro':[listaAtributos[3].get()] ,'Email': [listaAtributos[4].get()],'FechaCarnet':[listaAtributos[5].get()],'FechaNacimiento': [listaAtributos[6].get()],'TipoCarnet':[listaAtributos[7].get()] ,'CPConduccion':[listaAtributos[8].get()],'Cilindrada':[listaAtributos[9].get()],'ConductorUnico':[listaAtributos[10].get()],'Marca':[listaAtributos[11].get()],'Modelo':[listaAtributos[12].get()],'VehiculoHistorico':[listaAtributos[13].get()],'FechaMatriculacion':[listaAtributos[14].get()],'AccidentesConductor':[listaAtributos[15].get()],'Tipo de Pago':[listaAtributos[16].get()],'Importe':[listaAtributos[17].get()],'ModalidadSeleccionada':[listaAtributos[18].get()],'ClaseVehiculo':[listaAtributos[19].get()]}
    
    dataS = pd.DataFrame(data= dataSolicitado)  
    dataC = fn.cleanSolicitar(dataS)
    
    predNeuronal = mlp.predict_proba(dataC)
    predForest = randomForest.predict_proba(dataC)
    
    #Redes Neuronales
    redesNeuronalesLabel=Label(ventanaResultados, text="Redes neuronales", font=("arial bold", 14), anchor=CENTER,pady=6)
    noSiniestroLabelRRSS=Label(ventanaResultados, text="   - No siniestro: " + str(round(predNeuronal[0][0]*100, 2)) + " %" , font=("arial bold", 10), anchor=CENTER,pady=3)
    reclamacionLabelRRSS=Label(ventanaResultados, text="   - Reclamacion: " + str(round(predNeuronal[0][1]*100, 2)) + " %", font=("arial bold", 10), anchor=CENTER,pady=3)
    culpableLabelRRSS=Label(ventanaResultados,    text="   - Culpable: " + str(round(predNeuronal[0][2]*100, 2)) + " %" , font=("arial bold", 10), anchor=CENTER,pady=3)
    
    redesNeuronalesLabel.pack()
    noSiniestroLabelRRSS.pack()
    reclamacionLabelRRSS.pack()
    culpableLabelRRSS.pack()
    
    ventanaEspacio=Label(ventanaResultados,text="", font=("arial bold", 10), anchor=CENTER, pady=10)
    ventanaEspacio.pack()
    
    #Random Forest
    randomForestLabel=Label(ventanaResultados,  text="Random Forest" , font=("arial bold", 14), anchor=CENTER,pady=6)
    noSiniestroLabelRF=Label(ventanaResultados, text="   - No siniestro: " + str(round(predForest[0][0]*100, 2)) + " %" , font=("arial bold", 10), anchor=CENTER,pady=3)
    reclamacionLabelRF=Label(ventanaResultados, text="   - Reclamacion: " + str(round(predForest[0][1]*100, 2)) + " %" , font=("arial bold",10), anchor=CENTER,pady=3)
    culpableLabelRF=Label(ventanaResultados,    text="   - Culpable: " + str(round(predForest[0][2]*100, 2)) + " %" , font=("arial bold", 10), anchor=CENTER,pady=3)
    
    randomForestLabel.pack()
    noSiniestroLabelRF.pack()
    reclamacionLabelRF.pack()
    culpableLabelRF.pack()    
    
    #creamos button de Inicio y lo configuramos a nuestro gusto
    buttonInicio = Button(ventanaResultados, text = "Inicio", bg = "green",fg = "white",font = arial, command=lambda:ventanaInicio(ventanaResultados))
    buttonInicio.config(anchor=CENTER)
    
    buttonInicio.place(x=660,y=430)

## Ventana de inicio al pulsar el botón "Inicio"

In [ ]:
#Cuando pulsemos el boton de inicio queremos volver a la ventanaRaiz del programa

def ventanaInicio(ventana):
    destruirVent(ventana)
    ventanaInicial(raiz)
    
    

# Método mainloop

In [ ]:
raiz.mainloop() #Método que necesita estar corriendo continuamente para que la interfaz no pare de ser visible.